In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
batch_size = 32
img_height = 192
img_width = 192
data_dir = "./train_data_192px"

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=3434,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=3434,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_ds.class_names
print(class_names)

Found 4420 files belonging to 4 classes.
Using 3536 files for training.
Found 4420 files belonging to 4 classes.
Using 884 files for validation.
['angry', 'dsleepy', 'happy', 'sad']


In [12]:
model = Sequential(
    [
        layers.Rescaling(1.0 / 255, input_shape=(img_height, img_width, 3)),
        layers.RandomFlip("horizontal"),
        layers.Conv2D(16, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(4),
    ]
)

In [13]:
model.compile(optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],)

callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=0)
]

In [14]:
history = model.fit(train_ds, validation_data=val_ds, epochs=3, callbacks=callbacks)

Epoch 1/3


111/111 [==============================] - 111s 988ms/step - loss: 0.3322 - accuracy: 0.8945 - val_loss: 0.0052 - val_accuracy: 0.9989
Epoch 2/3
111/111 [==============================] - 111s 999ms/step - loss: 6.5459e-04 - accuracy: 1.0000 - val_loss: 7.3540e-04 - val_accuracy: 1.0000
Epoch 3/3
111/111 [==============================] - 112s 1s/step - loss: 3.0224e-04 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 0.9989


In [15]:
test_loss, test_accuracy = model.evaluate(val_ds)
print(f"Test accuracy: {test_accuracy}")

28/28 [==============================] - 6s 208ms/step - loss: 0.0015 - accuracy: 0.9989
Test accuracy: 0.9988687634468079


In [16]:
model.save('my_camera_model192_dataug2.h5')

c:\Users\c401\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
